# When is sensor moving
This notebook will go through the GPS data associated with each lidar frame to find the best way to tell when the sensor is moving or still, so I can make sets of consecutive stationary frames.

## Import libraries

In [39]:
import pandas as pd
import numpy as np
import os

## Read data from files

In [40]:
def get_data_from_dir(directory):
    data = []
    # Loop through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            # Extract the numeric part of the filename and convert it to integer
            file_index = int(filename.split('.')[0])  # Removes the extension and converts to int
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                values = file.read().strip().split()
                values = [float(value) for value in values]  # Convert each value to float
                data.append([file_index] + values)  # Add file index and values to data list
    return data

In [41]:
# Specify the directory where your .txt files are stored
stationary_directory = '../data/2011_09_26_drive_0017_sync/oxts/data'
moving_directory = '../data/2011_09_26_drive_0001_sync/oxts/data'

In [42]:
# Initialize a list to store data
stationary_data = get_data_from_dir(stationary_directory)
moving_data = get_data_from_dir(moving_directory)

## Make DataFrame

In [43]:
# Define column names (add 'file_name' for the filename column)
columns = [
    'file_name', 'lat', 'lon', 'alt', 'roll', 'pitch', 'yaw', 
    'vn', 've', 'vf', 'vl', 'vu', 'ax', 'ay', 'az', 'af', 
    'al', 'au', 'wx', 'wy', 'wz', 'wf', 'wl', 'wu', 
    'pos_accuracy', 'vel_accuracy', 'navstat', 'numsats', 
    'posmode', 'velmode', 'orimode'
]

# Create DataFrame
stationary_df = pd.DataFrame(stationary_data, columns=columns)
moving_df = pd.DataFrame(moving_data, columns=columns)

# Display the DataFrame
print(stationary_df.head())

   file_name        lat       lon         alt      roll     pitch       yaw  \
0         17  49.025527  8.448562  113.097458 -0.012579  0.013543  1.287138   
1         83  49.025527  8.448563  113.122055 -0.012359  0.013198  1.286996   
2         43  49.025527  8.448563  113.126770 -0.012155  0.013444  1.286256   
3         59  49.025527  8.448563  113.137512 -0.012132  0.013231  1.286442   
4         62  49.025527  8.448563  113.137505 -0.012195  0.013256  1.286513   

         vn        ve        vf  ...        wf        wl        wu  \
0  0.003869 -0.010586  0.000752  ... -0.000940  0.000076 -0.000697   
1 -0.007493 -0.004427 -0.008433  ...  0.000376 -0.000549  0.000601   
2 -0.003148  0.000545 -0.002868  ...  0.000618  0.000482 -0.000267   
3 -0.006582  0.002403 -0.005644  ...  0.000383 -0.000576  0.000945   
4 -0.007316  0.001881 -0.006495  ... -0.000430 -0.000230  0.000004   

   pos_accuracy  vel_accuracy  navstat  numsats  posmode  velmode  orimode  
0      0.054818      0.0127

In [44]:
stationary_df.describe()

,file_name,lat,lon,alt,roll,pitch,yaw,vn,ve,vf,...,wf,wl,wu,pos_accuracy,vel_accuracy,navstat,numsats,posmode,velmode,orimode
count,114.000000,1.140000e+02,1.140000e+02,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,114.000000,...,114.000000,114.000000,114.000000,114.000000,114.000000,114.0,114.000000,114.000000,114.000000,114.0
mean,56.500000,4.902553e+01,8.448563e+00,113.114068,-0.012359,0.013349,1.286815,-0.004031,-0.004613,-0.005160,...,0.000038,-0.000037,0.000004,0.054974,0.012798,4.0,12.719298,5.500000,5.500000,6.0
std,33.052988,1.241995e-07,1.675190e-07,0.015878,0.000166,0.000147,0.000398,0.003571,0.004582,0.002996,...,0.000420,0.000574,0.000527,0.002142,0.000215,0.0,0.451326,0.502208,0.502208,0.0
min,0.000000,4.902553e+01,8.448562e+00,113.084732,-0.012747,0.013131,1.286240,-0.009834,-0.011355,-0.010301,...,-0.001095,-0.001453,-0.001320,0.052802,0.012728,4.0,12.000000,5.000000,5.000000,6.0
25%,28.250000,4.902553e+01,8.448562e+00,113.101538,-0.012471,0.013226,1.286451,-0.007358,-0.009173,-0.007871,...,-0.000199,-0.000487,-0.000315,0.053413,0.012728,4.0,12.000000,5.000000,5.000000,6.0
50%,56.500000,4.902553e+01,8.448563e+00,113.112511,-0.012332,0.013303,1.286842,-0.004563,-0.004676,-0.004829,...,0.000065,-0.000009,0.000029,0.054203,0.012728,4.0,13.000000,5.500000,5.500000,6.0
75%,84.750000,4.902553e+01,8.448563e+00,113.128807,-0.012219,0.013469,1.286993,-0.001644,-0.000516,-0.002822,...,0.000348,0.000346,0.000367,0.054818,0.012728,4.0,13.000000,6.000000,6.000000,6.0
max,113.000000,4.902553e+01,8.448563e+00,113.137680,-0.012102,0.013750,1.287736,0.005493,0.002771,0.002888,...,0.000878,0.001935,0.001199,0.060308,0.013454,4.0,13.000000,6.000000,6.000000,6.0


In [45]:
moving_df.describe()

,file_name,lat,lon,alt,roll,pitch,yaw,vn,ve,vf,...,wf,wl,wu,pos_accuracy,vel_accuracy,navstat,numsats,posmode,velmode,orimode
count,108.00000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,...,1.080000e+02,108.000000,108.000000,108.000000,108.000000,108.0,108.000000,108.0,108.0,108.0
mean,53.50000,49.014762,8.433552,116.464296,0.029971,0.012445,-2.718472,-4.139566,-8.796198,9.733675,...,9.159165e-07,0.000626,-0.013340,0.027425,0.012171,4.0,9.842593,6.0,6.0,6.0
std,31.32092,0.000119,0.000399,0.017301,0.005170,0.003655,0.044789,1.373708,1.979072,2.357728,...,1.136685e-02,0.009207,0.013656,0.000619,0.000585,0.0,1.305359,0.0,0.0,0.0
min,0.00000,49.014596,8.432972,116.426804,0.019599,0.005031,-2.754572,-6.811441,-11.275642,4.611723,...,-4.124870e-02,-0.022495,-0.040925,0.026907,0.011314,4.0,7.000000,6.0,6.0,6.0
25%,26.75000,49.014658,8.433194,116.452316,0.026415,0.009947,-2.751804,-5.158587,-10.385950,7.963623,...,-7.241098e-03,-0.004362,-0.024349,0.026907,0.011314,4.0,9.000000,6.0,6.0,6.0
50%,53.50000,49.014744,8.433510,116.466743,0.030645,0.012657,-2.746841,-4.016737,-9.507683,10.321101,...,7.724918e-04,-0.000164,-0.011143,0.026907,0.012042,4.0,11.000000,6.0,6.0,6.0
75%,80.25000,49.014857,8.433890,116.479881,0.033122,0.013941,-2.690648,-3.053240,-7.355096,11.596100,...,6.308839e-03,0.005721,0.000941,0.027659,0.012728,4.0,11.000000,6.0,6.0,6.0
max,107.00000,49.015004,8.434297,116.490036,0.043410,0.021188,-2.608707,-1.803772,-4.244788,13.172717,...,4.485511e-02,0.031626,0.005523,0.028425,0.012728,4.0,11.000000,6.0,6.0,6.0


## Look at velocity data
vn:    velocity towards north (m/s)<br>
ve:    velocity towards east (m/s)<br>
vf:    forward velocity, i.e. parallel to earth-surface (m/s)<br>
vl:    leftward velocity, i.e. parallel to earth-surface (m/s)<br>
vu:    upward velocity, i.e. perpendicular to earth-surface (m/s)<br>

In [46]:
# Select the velocity columns
velocity_columns = ['vn', 've', 'vf', 'vl', 'vu']

In [47]:
# Calculate basic statistics for each velocity column
stationary_velocity_stats = stationary_df[velocity_columns].describe()

print(stationary_velocity_stats)

               vn          ve          vf          vl          vu
count  114.000000  114.000000  114.000000  114.000000  114.000000
mean    -0.004031   -0.004613   -0.005160   -0.003298   -0.000647
std      0.003571    0.004582    0.002996    0.004979    0.004490
min     -0.009834   -0.011355   -0.010301   -0.011792   -0.009167
25%     -0.007358   -0.009173   -0.007871   -0.007794   -0.003995
50%     -0.004563   -0.004676   -0.004829   -0.002903    0.000883
75%     -0.001644   -0.000516   -0.002822    0.001051    0.002375
max      0.005493    0.002771    0.002888    0.004256    0.008969


In [48]:
# Calculate basic statistics for each velocity column
moving_velocity_stats = moving_df[velocity_columns].describe()

print(moving_velocity_stats)

               vn          ve          vf          vl          vu
count  108.000000  108.000000  108.000000  108.000000  108.000000
mean    -4.139566   -8.796198    9.733675   -0.072965   -0.001610
std      1.373708    1.979072    2.357728    0.047267    0.027998
min     -6.811441  -11.275642    4.611723   -0.157731   -0.044877
25%     -5.158587  -10.385950    7.963623   -0.104715   -0.026471
50%     -4.016737   -9.507683   10.321101   -0.076025   -0.003085
75%     -3.053240   -7.355096   11.596100   -0.028728    0.016564
max     -1.803772   -4.244788   13.172717    0.016331    0.068896
